In [19]:
library(geometry)
library(openxlsx)

normalize_vec <- function(v,.min = NULL,.max=NULL)
{
  if (is.null(.min)) a <- min(v) else a <- .min
  if (is.null(.max)) b <- max(v) else b <- .max

  return( (v-a)/(b-a) )
}


normalize_df <- function(df,.Min=NULL,.Max=NULL)
{
  nc <- ncol(df)
  if (is.null(.Min)) .Min <- rep(NULL,nc)  
  if (is.null(.Max)) .Max <- rep(NULL,nc)  
    
  ret = df
  for (i in 1:nc)
  {
    ret[i] <- normalize_vec(df[i],.min=.Min[i],.max=.Max[i])
  }
  return(ret)
}

colMins <- function(df)
{
    nc <- ncol(df)
    ret<-rep(NULL,nc)
    
    for (i in 1:nc)
        ret[i] <- min(df[i])
    return(ret)
}

colMaxs <- function(df)
{
    nc <- ncol(df)
    ret<-rep(NULL,nc)
    
    for (i in 1:nc)
        ret[i] <- max(df[i])
    return(ret)    
}

In [20]:
method_file_name <- c("k_means", "random","som", "kenndard_stone")
data_file_path <- paste("data/",method_file_name,".xlsx",sep="")
data_file_path

[1] "data/k_means.xlsx"        "data/random.xlsx"        
[3] "data/som.xlsx"            "data/kenndard_stone.xlsx"

In [21]:
features <- c('Conc', 'SA', 'V', 'SA/V', 'Th-SC', 'Th-Ep', 'Th-T', 'Fol-Den', 'Fol-Dia', 'Enh', 'Kow', 
              'ConAng', 'Kow-sw', 'ConAng-sw')
features_reduced <- c('SA/V','Th-SC','Th-Ep','Th-T','Fol-Den','Fol-Dia','Kow','ConAng','Kow-sw')

label <- 'Layer'
labelQ <- 'LayerQ'

In [67]:
df_tv_list <- list()

for (i in 1:length(method_file_name))
{
      train_df <- read.xlsx(data_file_path[i], sheet = 1) # training dataset
      validation_df <- read.xlsx(data_file_path[i], sheet = 2) # validation dataset
      test_df <- read.xlsx(data_file_path[i], sheet = 3) # testing dataset 
    
     #removing the label columns
     train_df[label] <- NULL
     validation_df[label] <- NULL
     test_df[label] <- NULL
    
     train_df[labelQ] <- NULL
     validation_df[labelQ] <- NULL
     test_df[labelQ] <- NULL
    
     
    #get the unique observations and bind the dfs
    df_tv_list[[i]] <- unique(rbind(train_df,validation_df))[,features_reduced]       
    #df_tv_list[[i]] <- unique(train_df)[,features_reduced] 
    #df_tv_list[[i]] <- unique(validation_df)[,features_reduced] 
    #df_tv_list[[i]] <- unique(test_df)[,features_reduced] 
}

df_all <- unique(rbind(train_df,validation_df,test_df))[,features_reduced]

In [54]:
nc <- length(features_reduced)
minimums_all <- colMins(df_all)
maximums_all <- colMaxs(df_all)

minimums_all
maximums_all

[1]  0.03  6.18 11.73  0.70  0.00 25.00 -3.57  0.00  0.00

[1]   1.50000  17.82125  67.54286   2.54742 366.50000 200.00000   2.59000
[8] 151.00000   1.00000

In [68]:
#finding minimums and maximus of all observations, including testing set
minimums <- list()
maximums <- list()

for (i in 1:length(method_file_name))
{
    minimums[[i]] <- colMins( df_tv_list[[i]] )
    maximums[[i]] <- colMaxs( df_tv_list[[i]] )
}

**Aplpicability domain for the whole dataset**

In [56]:
convex_hull <- convhulln(as.matrix( normalize_df(df_all,minimums_all,maximums_all) ) ,"FS TO 'volume.txt'");
qh.outp <- scan("volume.txt")
QH_volume_all<- qh.outp[4] 
QH_volume_all

[1] 1.946005e-05

In [69]:
QH_volumes <- list()
for (i in 1:length(method_file_name))
{
    convex_hull <- convhulln(as.matrix( normalize_df(df_tv_list[[i]],minimums_all,maximums_all) ) ,"FS TO 'volume.txt'");
    qh.outp <- scan("volume.txt")    
    QH_volumes[method_file_name[i]] <- qh.outp[4] 
}
QH_volumes

$k_means
[1] 1.432342e-05

$random
[1] 1.432342e-05

$som
[1] 1.432342e-05

$kenndard_stone
[1] 1.432342e-05